In [2]:
!pip install -q transformers datasets
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.2 MB/s eta 0:00:00


In [3]:
import numpy as np

In [4]:
dataset = load_dataset("LucasThil/miniwob_actions_onhot")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/652385 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/81548 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/81549 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_actions_onhot-c67c0ff2855c5532/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
example = dataset['train']['episodes'][3]
label_example = dataset['train']['refs'][3]
print(f'label: {label_example}')
example

label: 15


'{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{

In [11]:
# We need to identify the salient elements of the page, for that we parse all actions from the context
def unique(list1):
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def get_actions_refs(action_parts):
  actions_refs = []
  for entry in action_parts:
    entry_parts = entry.split(', ')
    action_part = entry_parts[0]
    action = action_part.replace('action: ', '').replace('{', '')
    if len(entry_parts) < 2:
      #print(entry_parts)
      continue
    ref_part = entry_parts[1]
    ref = ref_part.replace('ref: ', '')

    actions_refs.append([action, ref])

  return actions_refs

def get_parsed_html(html):
  html = html.replace('><', '>ø<').split('ø')
  return html

def preprocess_entry(text):
  # part 0 are actions, parts len(parts)-1 is html
  parts = text.split("} ")
  if (len(parts)) < 2 and False:
    print(f'got wrong amount of parts: {len(parts)}, for text {text}')
  action_parts = parts[0].split('}{')
  actions_refs = get_actions_refs(action_parts)

  html = parts[len(parts)-1]
  print(f'html: {html}')
  html = get_parsed_html(html)
  return actions_refs, html

def append_html_depth(html, depth):
  html_depths = []
  children = []
  i = 0
  
  # get current element
  #element = html[0]

  while(i<len(html)):
    element = html[i]
    # if start tag
    if '</' not in element:
      
      # recursion to go deeper
      children.append(element)
      found_children, found_depth, found_i = append_html_depth(html[i+1:], depth+1)
      children.append(found_children)
      i += found_depth
      #i += found_i
    else:
      # if <! closing tag
      #print(element)
      if i==0:
        if '</' in html[i+1]:
          return element, depth, i
      else:
        #children.append(element)
        return children, depth+1, i
  
  return children, depth, i


def append_html_depth2(html, depth, ref_id, found_refs):
  children = []

  i = 0
  while i < len(html):
    element = html[i]
    # if opening tag <...
    #if '</' not in element:
    #  # go deeper

    if remove_forbidden_tag(element):
      i += 1
      continue


    if not '</' in element:

      found_ref_nb = find_ref_element(element)

      element = remove_class(element)
      element = element.replace('>', '')
      # add our own ref number
      element += 'ref="' + str(ref_id) + '">'
      # old_id to new_id:
      if found_ref_nb is not None:
        found_refs[found_ref_nb] = ref_id
      ref_id += 1 # increase count of ref_id for that element

      children.append(element)
      #print(f'1 {element} {children} i={i}, depth={depth}')
      # another opening tag, go deeper
      found_children, nb_found_elements, ref_id, found_refs_deep = append_html_depth2(html[i+1:], depth+1, ref_id, found_refs)
      children.append(found_children)
      i += nb_found_elements
      if type(found_children) is type(list()) and len(found_children) > 0 and i+1 < len(html):
        # we need to add the element here
        #print(f'put closer {html[i]}, {found_children}')
        children.append(html[i])
        

        

        #i+=1

      # add the ref_id to the dict:
      #found_refs = found_refs_deep # update with the one found deeper
      for key in found_refs_deep.keys():
        found_refs[key] = found_refs_deep[key]
      
        
    else:
      #print(f'SPECIAL {element} {len(html)}, {i}')
      if len(children) == 0:
        #print(f'2 {element} {children} i={i}, depth={depth}')
        return element, i+1, ref_id, found_refs
      #elif i+1 < len(html) and '</' in html[i+1]:
      #  #children.append(element)
      #  print(f'3 {element} {children} i={i}, depth={depth}')
      #  return children, i+1
      else:
        #print(f'4 {element} {children} i={i}, depth={depth}')
        #children.append(element)
        return children, i+1, ref_id, found_refs


      ##if i+1<len(html) and '</' in html[i+1]:
      #  if len(children) == 1:
      #    print(f'3 {children} i={i}, depth={depth}')
      #    return element, i+1
      #  else:
      #    print(f'4 {children} i={i}, depth={depth}')
      #    return children i+1


      #if i == 0:
      #  print(f'3 {element} i={i}, depth={depth}')
      #  return element, i+1
      #else:
      #  # we close at that level
      #  print(f'2 {children} i={i}, depth={depth}')
      #  children.append(element)
      #  #i += 1
      #  return children, i+1
      
     
    #else:
    #  # Closing tag, if followed by anohter one return
    #  print(f'2 {element} i={i} depth={depth}')
    #  #if '</' in html[i+1]:
    #  children.append(element)
    #  return children, i+1
    #  #else:
    #    # following element is not a closing tag, thus remain at the same level
    #  #children.append(element)
    
    i += 1

  #print('4')
  children.append(html[i:])
  return children, i, ref_id, found_refs


# Indicates whether the element should be discarded
def remove_forbidden_tag(element):
  element = element.lower()
  to_remove = ['<svg', '<path', '<img', '<iframe', '</svg', '</path', '</img', '</iframe' ]
  for tag in to_remove:
    if tag in element:
      #print(f'remove_forbidden_tag {element}')
      return True
  return False

# returns the found ref in the element
def find_ref_element(element):
  if 'ref=' in element:
    el = ""
    parts = element.split(' ')
    i = 0
    while i < len(parts):
      if 'ref=' in parts[i]:
        ref_p = parts[i].replace('ref=', '').replace('"', "").replace('>', "")
        #print(f'found ref: {ref_p}')
        #ref_nb = parts[i+1]
        return ref_p
      i += 1
  None        
        

# Deletes special fields in the element such as tag
# Delete the initial ref, we'll put these values ourselves
def remove_class(element):
  element = element.lower()
  to_remove = ['class', 'ref']

  for tag in to_remove:
    if tag in element:
      #print(f'remove_class {element}')
      el = ""
      parts = element.split(' ')
      for p in parts:
        if not tag in p:
          el += p + ' '
      element = el
  return element



def print_nice_html(parsed_html, depth):

  for entry in parsed_html:
    if type(entry) != type(list()):
      print('    ' * depth + '' + entry + ', ' + str(depth))
    else:
      print_nice_html(entry, depth+1)
    

In [15]:
actions_refs, html = preprocess_entry(example)

print(f'actions_refs: {actions_refs}')
html

html: <BODY ref="1"><A classes="banner" ref="2"><IMG classes="alaska_logo" ref="3"></IMG></A><DIV id="main" ref="4"><DIV id="starth1" classes="header" ref="5"><H1 classes="h1" ref="6" text="Book a flight"></H1></DIV><FORM id="searchForm" classes="miniwob-main-form" ref="7"><DIV classes="form-row group" ref="8"><DIV classes="left" ref="9"><LABEL classes="css-lbl" ref="10" text="One-way"></LABEL></DIV><DIV id="use-miles-div" classes="right" ref="11"><LABEL classes="css-lbl" ref="12" text="Use miles"></LABEL></DIV></DIV><DIV id="geo-from-wrap" classes="form-row geo-wrap" ref="13"><LABEL classes="text-label" ref="14"><t ref="-149" text="From"></t><INPUT_text id="geo-from" classes="text-input-pad " ref="15" recordingTarget="True" value="altantic ci"></INPUT_text></LABEL><DIV classes="cleartxt" ref="16"></DIV><DIV id="geo-from-button" classes="geo-button" ref="17"><SPAN classes="geo-img" ref="18" text="Geolocation"></SPAN></DIV></DIV><DIV id="geo-to-wrap" classes="form-row geo-wrap" ref="19"

['<BODY ref="1">',
 '<A classes="banner" ref="2">',
 '<IMG classes="alaska_logo" ref="3">',
 '</IMG>',
 '</A>',
 '<DIV id="main" ref="4">',
 '<DIV id="starth1" classes="header" ref="5">',
 '<H1 classes="h1" ref="6" text="Book a flight">',
 '</H1>',
 '</DIV>',
 '<FORM id="searchForm" classes="miniwob-main-form" ref="7">',
 '<DIV classes="form-row group" ref="8">',
 '<DIV classes="left" ref="9">',
 '<LABEL classes="css-lbl" ref="10" text="One-way">',
 '</LABEL>',
 '</DIV>',
 '<DIV id="use-miles-div" classes="right" ref="11">',
 '<LABEL classes="css-lbl" ref="12" text="Use miles">',
 '</LABEL>',
 '</DIV>',
 '</DIV>',
 '<DIV id="geo-from-wrap" classes="form-row geo-wrap" ref="13">',
 '<LABEL classes="text-label" ref="14">',
 '<t ref="-149" text="From">',
 '</t>',
 '<INPUT_text id="geo-from" classes="text-input-pad " ref="15" recordingTarget="True" value="altantic ci">',
 '</INPUT_text>',
 '</LABEL>',
 '<DIV classes="cleartxt" ref="16">',
 '</DIV>',
 '<DIV id="geo-from-button" classes="geo-

In [13]:
parsed_html, _, ref_id, found_refs = append_html_depth2(html, 0, 1, {})
#print(html)
print(parsed_html)
print_nice_html(parsed_html, 0)
print(ref_id)
# The following dict maps the old refs to the new ones we have
print(found_refs)


# Change: new translation of the action history:
action_sequence = ""
for ac in actions_refs:
  act_seq = "{" + ac[0] + ',' + str(found_refs[ac[1]]) + '}'
  action_sequence += act_seq
print(action_sequence)

# Iterate through actions_refs and update them with the new values
print(f'actions_refs: {actions_refs}')
new_actions_refs = []
for e in actions_refs:
  new_actions_refs.append([e[0], str(found_refs[e[1]]) ])
print(f'new_actions_refs: {new_actions_refs}')
actions_refs = new_actions_refs
print(f'found_refs: {found_refs}')

['<body ref="1">', ['<a  ref="2">', '</A>', '<div id="main" ref="3">', ['<div id="starth1"  ref="4">', ['<h1 text="book a flight"  ref="5">', '</H1>'], '</DIV>', '<form id="searchform"  ref="6">', ['<div group"  ref="7">', ['<div  ref="8">', ['<label text="one-way"  ref="9">', '</LABEL>'], '</DIV>', '<div id="use-miles-div"  ref="10">', ['<label text="use miles"  ref="11">', '</LABEL>'], '</DIV>'], '</DIV>', '<div id="geo-from-wrap" geo-wrap"  ref="12">', ['<label  ref="13">', ['<t text="from" ref="14">', '</t>', '<input_text id="geo-from" " recordingtarget="true" value="altantic ci"  ref="15">', '</INPUT_text>'], '</LABEL>', '<div  ref="16">', '</DIV>', '<div id="geo-from-button"  ref="17">', ['<span text="geolocation"  ref="18">', '</SPAN>'], '</DIV>'], '</DIV>', '<div id="geo-to-wrap" geo-wrap"  ref="19">', ['<label  ref="20">', ['<t text="to" ref="21">', '</t>', '<input_text id="geo-to"  ref="22">', '</INPUT_text>'], '</LABEL>', '<div  ref="23">', '</DIV>', '<div id="geo-to-button"

In [ ]:
# define function to get snippets
# TODO NOW: use 'found_refs' dict from above to update the actions below

# parsed_html
actions_refs = np.array(actions_refs)

print(f'actions_refs {actions_refs}')
if len(actions_refs) == 0:
  unique_refs = []
else:
  unique_refs = list(set(actions_refs[:, 1]))

# NEW: translate the target_label
print(f'label_example: {label_example}')
label_example = found_refs[str(label_example)]
print(f'new label_example: {label_example}')

# add target label to the array of unique_refs
if str(label_example) not in unique_refs:
  unique_refs.append(str(label_example))
print(unique_refs)

# create a dictionary to check whether we found the indices of each element when creating the snippet
dict_refs = {}
for el in unique_refs:
  dict_refs.update({el: False})

print(dict_refs)

actions_refs [['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']
 ['keyup' '15']
 ['keypress' '15']
 ['keypress' '15']
 ['keydown' '15']
 ['keydown' '15']
 ['keyup' '15']

In [ ]:

def heuristic(refs, depth):
  all_true = all(value for value in refs.values())
  
  #if not all_true:
  #print(refs)
  #print(f'test heuristic depth {depth} {refs}')

  return all_true

# iterate through the vectorized html elements to find the location of the element with the ref
def find_ref_element_html(refs, htmls, depth, metrics_dict, snippet):
  
  i = 0
  pre_snippet = snippet

  while i < len(htmls):
    
    element = htmls[i]

    if type(element) == type(list()):
      flag, new_snippet, metrics_dict = find_ref_element_html(refs, element, depth+1, metrics_dict, pre_snippet)
      if flag:
        # found good code down there so add it
        snippet = new_snippet
        if heuristic(refs, depth):
            return True, snippet, metrics_dict
    else:
      pre_snippet += element
      print(f'{depth} - {pre_snippet}')

      for key in refs.keys():
        if str('ref="' + key + '"') in element:
          refs[key] = True

          # found first ref value, we know where to start
          if metrics_dict['found_first'] == False:
            metrics_dict['found_first'] = True
            metrics_dict['min_depth'] = depth

          # If previous min depth if larger than the new one, update
          if metrics_dict['min_depth'] > depth and metrics_dict['found_first'] == True:
            metrics_dict['min_depth'] = depth

          # update max one
          if metrics_dict['max_depth'] <= depth:
            metrics_dict['max_depth'] = depth

          print(f'found at depth {depth} {element}')
          snippet = pre_snippet

          if heuristic(refs, depth):
            return True, snippet, metrics_dict
          

    i += 1
  
  # No final elements found, return false
  return heuristic(refs, depth), snippet, metrics_dict


# New function to generate the snippets:
# Iterate through the elements of that depth.
# When a list element is found, recurse 
def generate_snippet2(refs, htmls, depth):

  i = 0;  
  level_snippet = "" # snippet collected at that level and below
  flag_level = False # If we found an element at that level or below return a True flag

  while i < len(htmls):

    element = htmls[i]


    if type(element) == type(list()):
      # Go deeper
      found_flag_level, found_level_snippet = generate_snippet2(refs, element, depth+1)

      if found_flag_level:
        # New elements were found, thus append them to current level snippet
        level_snippet += found_level_snippet
        flag_level = True

    else:
      # Element present at our level
      # Ok: maybe add all elements of that level if just one of them has a ref
      sub_key_flag = False # Checks whether a key is found below to avoid reappending twice
      for key in refs.keys():
        if str('ref="' + key + '"') in element:
          # key ref is in element so add it there
          level_snippet += element
          # Update flag has we have found a new element
          flag_level = True
          sub_key_flag = True
        
      # If we found an element at that level, add following ones for diversity
      # keep param i<2 to avoid it being too large
      if (not flag_level and not sub_key_flag and i < 5) or depth==0:
        level_snippet += element


    i +=1

    
  # all elements at that level and below have been explored, return
  # should return a boolean whether at least one new element needed to be found has been found
  return flag_level, level_snippet




# For the heuristic, we create a dict to keep track of certain values
# Also define hyperparams
metrics_dict = {
    'min_depth': 0,
    'max_depth': 0,
    'found_first': False,

    # hyper params
    'param_max_depth': 7,
    'max_children': 15,
    'max_length': 512
}

# create a dictionary to check whether we found the indices of each element when creating the snippet
dict_refs = {}
for el in unique_refs:
  dict_refs.update({el: False})

print(f'dict_ref {dict_refs}')

snippet = ""

print_nice_html(parsed_html, 0)

#success, snippet, metrics = find_ref_element_html(dict_refs, parsed_html, 0, metrics_dict, snippet)
success, snippet, metrics = generate_snippet2(dict_refs, parsed_html, 0, metrics_dict)
print(f'succes:{success} len: {len(snippet)} snippet: {snippet}')

NameError: ignored

In [ ]:
print(len(example.split('} ')[0]))
print(example.split('} ')[0])

837
{action: mouseup, ref: 12}{action: mousedown, ref: 10}{action: mousedown, ref: 10}{action: click, ref: 10}{action: click, ref: 10}{action: mouseup, ref: 10}{action: mouseup, ref: 10}{action: mousedown, ref: 8}{action: mousedown, ref: 8}{action: click, ref: 8}{action: click, ref: 8}{action: mouseup, ref: 8}{action: mouseup, ref: 8}{action: mousedown, ref: 6}{action: mousedown, ref: 6}{action: click, ref: 6}{action: click, ref: 6}{action: mouseup, ref: 6}{action: mouseup, ref: 6}{action: mousedown, ref: 4}{action: mousedown, ref: 4}{action: click, ref: 4}{action: click, ref: 4}{action: mouseup, ref: 4}{action: mouseup, ref: 4}{action: mousedown, ref: 2}{action: mousedown, ref: 2}{action: click, ref: 2}{action: click, ref: 2}{action: mouseup, ref: 2}{action: mouseup, ref: 2}{action: mousedown, ref: 23}{action: mousedown, ref: 23


In [ ]:
print(len(snippet))
print(snippet)

410
<input type="checkbox" id="ch0" value="true" ref="6"></input><input type="checkbox" id="ch1" value="true" ref="9"></input><input type="checkbox" id="ch2" value="true" ref="12"></input><input type="checkbox" id="ch3" value="true" ref="15"></input><input type="checkbox" id="ch4" value="true" ref="18"></input><input type="checkbox" id="ch5" recordingtarget="true" value="false" ref="21"></input><label ref="39">


In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
encoding = tokenizer(example.split('} ')[0] + snippet, padding="max_length", truncation=True, max_length=512)
input_ids = encoding['input_ids']
print(f'length: {len(input_ids)}')
encoding

length: 512


{'input_ids': [101, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 2260, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 2184, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 1022, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1020, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1020, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 1020, 1065, 1063, 2895, 1024, 11562, 1010

# Prepare everything in a nice loop

In [ ]:
features = ['click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll']

def prepare_snippets(dataset, data_label):
  i = 0

  snippet_list = []
  target_ref_list = []
  one_hots = []
  failed_snippets = 0
  failed_actions = 0
  failed_targets = 0
  body_only_actions = 0

  for example in dataset[data_label]:
    #print(f'example: {example}')
    target_label = example['refs']

    example_one_hot = []
    for f in features:
      example_one_hot.append(example[f])

    example = example['episodes']
    #print(example)

    actions_refs, html = preprocess_entry(example)
    parsed_html, _, ref_id, found_refs = append_html_depth2(html, 0, 0, {})
    #print_nice_html(parsed_html, 0)
    #print(f'ref_id: {ref_id}')
    #print(f'found_refs: {found_refs}')
    #print(f'actions_refs: {actions_refs}')

    # For the heuristic, we create a dict to keep track of certain values
    # Also define hyperparams
    metrics_dict = {
        'min_depth': 0,
        'max_depth': 0,
        'found_first': False,
        # hyper params
        'param_max_depth': 7,
        'max_children': 15,
        'max_length': 512
    }

    # add target label tp unique_refs
    actions_refs = np.array(actions_refs)
    if len(actions_refs) == 0:
      unique_refs = []
    else:
      unique_refs = list(set(actions_refs[:, 1]))


    # Change: new translation of the action history:
    # + Iterate through actions_refs and update them with the new values
    action_sequence = ""
    new_actions_refs = []
    for ac in actions_refs:

      # Sometimes, html snippets do not have the ref so just avoid it
      try:
        #act_seq = "{" + ac[0] + ',' + ac[1] + '}'
        act_seq = "{" + ac[0] + ',' + str(found_refs[ac[1]]) + '}'
        new_actions_refs.append([ac[0], str(found_refs[ac[1]]) ])
        action_sequence += act_seq
      except:
        failed_actions += 1
        continue
    actions_refs = new_actions_refs


    # NEW: translate the target_label
    try:
      target_label = found_refs[str(target_label)]
    except:
      # If fail, the target element is not present here so we pass it
      print(f'_____________fail____________ target_label:{target_label}, found_refs:{found_refs}')
      i += 1
      failed_targets += 1
      continue

    # add target label to the array of unique_refs
    if str(target_label) not in unique_refs:
      unique_refs.append(str(target_label))
    #print(f'target_label: {target_label}')
    #print(f'unique_refs: {unique_refs}')

    # create a dictionary to check whether we found the indices of each element when creating the snippet
    dict_refs = {}
    for el in unique_refs:
      dict_refs.update({el: False})

    snippet = ""
    #success, snippet, metrics = find_ref_element_html(dict_refs, parsed_html, 0, metrics_dict, snippet)
    success, snippet = generate_snippet2(dict_refs, parsed_html, 0)

    # If target ref is not present, discard
    ##print(f'target_label: {target_label}, snippet: {snippet}')
    if 'ref="' + str(target_label) + '"' not in snippet:
      failed_snippets += 1
      print(f'{i} target_label NOT IN SNIPPET: {target_label}')
      continue

    if success and len(snippet) > 30 and str(target_label) != "0":# and snippet != '<body ref="1">':
      snippet = action_sequence + snippet
      snippet = snippet.replace("  ", " ")
      snippet_list.append(snippet)
      #print(f'snippet: {snippet}')

      # add target ref
      target_ref_list.append(target_label)

      # add one hots
      one_hots.append(example_one_hot)

    #elif snippet == '<body ref="1">':
      #print(f'got only body, dicars {snippet}')
      #body_only_actions += 1
    else:
      failed_snippets += 1
      print(f'error {success},  target_label={target_label}')
      #print(f'parsed_html: {parsed_html}')
      #break


    i += 1
    print(f'Entry {i} - snippets: {len(snippet_list)}, failed_snippets: {failed_snippets}, failed_actions: {failed_actions}, failed_targets: {failed_targets}')
    # stopper
    #if i > 10: #and False:
    #  break

  print(f'failed_actions: {failed_actions}, failed_targets: {failed_targets}')

  return snippet_list, failed_snippets, body_only_actions, target_ref_list, one_hots


In [ ]:
split = 'train'
train_snippets, failed_snippets, body_only_actions, target_ref_list, one_hots = prepare_snippets(dataset, split)
print(f' train_snippets: {len(train_snippets)}, failed snippets: {failed_snippets}, body_only_actions: {body_only_actions}')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
error True,  target_label=0
Entry 648453 - snippets: 461231, failed_snippets: 179076, failed_actions: 1370680, failed_targets: 8146
Entry 648454 - snippets: 461232, failed_snippets: 179076, failed_actions: 1370680, failed_targets: 8146
Entry 648455 - snippets: 461233, failed_snippets: 179076, failed_actions: 1370680, failed_targets: 8146
Entry 648456 - snippets: 461234, failed_snippets: 179076, failed_actions: 1370680, failed_targets: 8146
Entry 648457 - snippets: 461235, failed_snippets: 179076, failed_actions: 1370680, failed_targets: 8146
error True,  target_label=0
Entry 648458 - snippets: 461235, failed_snippets: 179077, failed_actions: 1370680, failed_targets: 8146
Entry 648459 - snippets: 461236, failed_snippets: 179077, failed_actions: 1370680, failed_targets: 8146
Entry 648460 - snippets: 461237, failed_snippets: 179077, failed_actions: 1370680, failed_targets: 8146
Entry 648461 - snippets: 461238, f

In [ ]:
# new snippets_test: 0.0 0.2762544758914995 57976 57976 57976 81548
# new snippets_validate 0.0 0.2762544758914995 57976 57976 57976 81548

# train: failed_actions: 1381249, failed_targets: 8185 train_snippets: 464060, failed snippets: 180140, body_only_actions: 0
# 0.27612529411313874   464060    652385
print(body_only_actions / len(dataset[split]))
# 5% are body element only actions
print(failed_snippets / len(dataset[split]))
print(len(train_snippets))
print(len(target_ref_list))
print(len(one_hots))
print(len(dataset[split]))

0.0
0.27612529411313874
464060
464060
464060
652385


In [ ]:
i = 0
while i < len(train_snippets):
  print(train_snippets[i])
  print(target_ref_list[i])
  print(one_hots[i])
  i += 1

  if i> 100:
    break

{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{click,14}{click,14}{mouseup,14}{mouseup,14}{mousedown,0}{mousedown,0}{scroll,0}<body ref="0"><a ref="1"><div id="starth1" ref="3"></DIV><form id="searchform" ref="5"><div group" ref="6"></DIV><div id="geo-from-wrap" geo-wrap" ref="11"><label ref="12"><t text="from" ref="13"></t><input_text id="geo-from" recordingtarget="true" ref="14"><div ref="15">
14
[0, 0, 1, 0, 0, 0, 0, 0]
{keypress,11}{keypress,11}{keydown,11}{keydown,11}{keydown,11}{keydown,11}{scroll,11}{click,11}{click,11}{mouseup,11}{mouseup,11}{mousedown,0}{mousedown,0}{click,0}{click,0}{mouseup,0}{mouseup,0}{mousedown,0}{mousedown,0}{click,0}{click,0}{mouseup,0}{mouseup,0}{mousedown,0}{mousedown,0}find the email by cathryn and forward that email to dari. <body ref="0"><form id="mturk_form" ref="1"><div id="wrap" ref="2"><div id="area" ref="3"><div id="forward

In [ ]:
import numpy as np
import pandas as pd

def combine_datasets(train_snippets, target_ref_list, one_hots, split):
  # combine dataset
  print(f'len actions{len(one_hots)}')
  train_data_episodes = train_snippets
  print(f'len episodes {len(train_data_episodes)}')
  train_data_refs = target_ref_list
  print(f'len refs {len(train_data_refs)}')

  actions = np.array(one_hots)

  data = {
    "episodes": train_data_episodes,
    "refs": train_data_refs,
    "click": actions[:,0].astype(np.int64),
    "dblclick": actions[:,1].astype(np.int64),
    "keydown": actions[:,2].astype(np.int64),
    "keypress" : actions[:,3].astype(np.int64),
    "keyup" : actions[:,4].astype(np.int64),
    "mousedown": actions[:,5].astype(np.int64),
    "mouseup": actions[:,6].astype(np.int64),
    "scroll": actions[:,7].astype(np.int64),
  }


  train_csv = pd.DataFrame(data)
  train_csv.to_csv("/content/sample_data/" + split + ".csv", index=False)

  return train_csv

In [ ]:
train_csv = combine_datasets(train_snippets, target_ref_list, one_hots, split)

len actions464060
len episodes 464060
len refs 464060


In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
  "test" : "/content/sample_data/test.csv",
  "validate": "/content/sample_data/validate.csv"
}
df_train = load_dataset("csv", data_files=file_dict)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validate split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4037924762a84c3b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
df_train.push_to_hub('LucasThil/miniwob_snippet_improved')

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/233 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/233 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/59 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/58 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import numpy as np
l = np.array([[0, 1, 2, 5]])
l

array([[0, 1, 2, 5]])

In [ ]:
l.ravel()

array([0, 1, 2, 5])

# Convert into One-Hot encodings

In [ ]:
dataset = load_dataset("LucasThil/miniwob_snippet_improved")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/464060 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/57976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/58068 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_snippet_improved-035c4290dfb38a83/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
chars_refs = sorted(list(set(dataset['train']['refs'] + dataset['test']['refs'] + dataset['validate']['refs'])))
chars_refs_size = len(chars_refs)
print(chars_refs)
print(chars_refs_size)

stoi = {ch:i for i, ch in enumerate(chars_refs)}
itos = {i:ch for i, ch in enumerate(chars_refs)}
print(stoi)
print(itos)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 166, 180, 181]
158
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 32: 30, 33: 31, 34: 32, 35: 33, 36: 34, 37: 35, 38: 36, 39: 37, 40: 38, 41: 39, 42: 40, 43

In [ ]:


def create_one_hot_refs(rows):
  refs = np.zeros((len(rows), len(chars_refs)))

  i = 0
  for row in rows['refs']:
    #sub_action = np.zeros(action_size)
    #sub_action[stoi[row]] = 1

    refs[i, stoi[row]] = 1

    i += 1

  return refs


In [ ]:
import pandas as pd

In [ ]:

actions = ['click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll']

def combine_datasets(dataset, refs, split):
  # combine dataset
  print(f'len actions {split} {len(actions)}')
  train_data_episodes = dataset[split]['episodes']
  print(f'len episodes {split} {len(train_data_episodes)}')
  train_data_refs = dataset[split]['refs']
  print(f'len refs {split} {len(train_data_refs)}')

  data = {
    "episodes": train_data_episodes,
    "refs": train_data_refs,
    "click": dataset[split]['click'],
    "dblclick": dataset[split]['dblclick'],
    "keydown": dataset[split]['keydown'],
    "keypress" : dataset[split]['keypress'],
    "keyup" : dataset[split]['keyup'],
    "mousedown": dataset[split]['mousedown'],
    "mouseup": dataset[split]['mouseup'],
    "scroll": dataset[split]['scroll'],
  }

  for ch in stoi:
    data[ch] = refs[:, stoi[ch]].astype(np.int64)

  train_csv = pd.DataFrame(data)
  print(train_csv.head())
  print(train_csv.columns)
  train_csv.to_csv("/content/sample_data/" + split + ".csv", index=False)

  return train_csv

In [ ]:
train_csv = combine_datasets(dataset, create_one_hot_refs(dataset['validate']), 'validate')

len actions validate 8
len episodes validate 57976
len refs validate 57976
                                            episodes  refs  click  dblclick  \
0  {mouseup,20}{mousedown,17}{mousedown,17}{click...    20      0         0   
1  {click,14}{click,0}{click,0}{mouseup,0}{mouseu...    14      1         0   
2  {keydown,5}{keydown,5}{keydown,5}{keydown,5}{k...     5      0         0   
3  {mouseup,17}{mousedown,36}{mousedown,36}{click...    17      0         0   
4  {keydown,4}{keyup,4}{keyup,4}{keypress,4}{keyp...     4      0         0   

   keydown  keypress  keyup  mousedown  mouseup  scroll  ...  152  153  154  \
0        0         0      0          0        1       0  ...    0    0    0   
1        0         0      0          0        0       0  ...    0    0    0   
2        1         0      0          0        0       0  ...    0    0    0   
3        0         0      0          0        1       0  ...    0    0    0   
4        1         0      0          0        0       0

In [ ]:
train_csv.head()

,episodes,refs,click,dblclick,keydown,keypress,keyup,mousedown,mouseup,scroll,...,152,153,154,155,156,157,160,166,180,181
0,"{mouseup,20}{mousedown,17}{mousedown,17}{click...",20,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"{click,14}{click,0}{click,0}{mouseup,0}{mouseu...",14,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"{keydown,5}{keydown,5}{keydown,5}{keydown,5}{k...",5,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"{mouseup,17}{mousedown,36}{mousedown,36}{click...",17,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"{keydown,4}{keyup,4}{keyup,4}{keypress,4}{keyp...",4,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
  "test" : "/content/sample_data/test.csv",
  "validate": "/content/sample_data/validate.csv"
}
df_train = load_dataset("csv", data_files=file_dict)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validate split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8a3e3c4a41528ace/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df_train.push_to_hub('LucasThil/miniwob_snippets_refs_onehot')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/155 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/155 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/59 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/58 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]